### setup

In [ ]:
from __future__ import division

import os
import urllib, cStringIO

import pymongo as pm
import numpy as np
import scipy.stats as stats
import pandas as pd
import json
import re
import ast

from PIL import Image
import base64
import sys

from IPython.display import clear_output

## plotting
import matplotlib
from matplotlib import pylab, mlab, pyplot
%matplotlib inline
from IPython.core.pylabtools import figsize, getfigs
plt = pyplot
import seaborn as sns
sns.set_context('talk')
sns.set_style('white')

from collections import Counter

## svg rendering 
# import ast
# from svgpathtools import parse_path, wsvg, svg2paths

#### paths etc.


In [ ]:
# directory & file hierarchy
proj_dir = os.path.abspath('../../..')
stimulus_dir = os.getcwd()
analysis_dir = os.path.join(proj_dir,'analysis')
results_dir = os.path.join(proj_dir,'results')
plot_dir = os.path.join(results_dir,'plots')
csv_dir = os.path.join(results_dir,'csv')
exp_dir = os.path.abspath(os.path.join(proj_dir,'experiments'))
sketch_dir = os.path.abspath(os.path.join(proj_dir,'sketches'))

## add helpers to python path
if os.path.join(proj_dir,'analysis','python') not in sys.path:
    sys.path.append(os.path.join(proj_dir,'analysis','python'))
    
if not os.path.exists(results_dir):
    os.makedirs(results_dir)
    
if not os.path.exists(plot_dir):
    os.makedirs(plot_dir)   
    
if not os.path.exists(csv_dir):
    os.makedirs(csv_dir)       
    
# Assign variables within imported analysis helpers
import df_generation_helpers as h
if sys.version_info[0]>=3:
    from importlib import reload
reload(h)

#### load in group data csv

In [ ]:
path_to_group_data = os.path.join(results_dir,'graphical_conventions.csv')
X = pd.read_csv(path_to_group_data)

## remove unnecessary columns
borked_cols = ['Unnamed: 0','row_index']
for this_col in borked_cols:
    if this_col in X.columns:
        X = X.drop(labels=[this_col], axis=1)    

### render sketches using svg data (can be skipped if already rendered)

In [ ]:
import svg_rendering_helpers as srh
### this is the order we want: ['run_num','gameID','trial_num','condition','target','repetition']

In [ ]:
## extract sketch identifying info
run_dict = {'within':'run3','between':'run4'}
runseries = [run_dict[i] for i in X['Generalization']]
gseries = X['gameID'].map(str)
nseries = X['trialNum'].map(str).apply(lambda x: x.zfill(2))
cseries = X['condition'].map(str)
tseries = X['target'].map(str)
rseries = X['repetition'].map(str).apply(lambda x: x.zfill(2))

## build list of image filenames
fname_list = ['{}_{}_{}_{}_{}_{}'.format(f,g,h,i,j,k) \
              for (f,g,h,i,j,k) in zip(runseries,gseries,nseries,cseries,tseries,rseries)]

## convert svg string strings into svg string list
svg_string_list = [ast.literal_eval(i) for i in X.svgString.values]

In [ ]:
## render out svg & convert to png
reload(srh)
reallyRun = 1
if reallyRun:
    for this_fname,this_svg in zip(fname_list,svg_string_list):    
        srh.render_svg(this_svg,base_dir=sketch_dir,out_fname= '{}.svg'.format(this_fname))    
        print 'Rendering {}'.format(this_fname)
        clear_output(wait=True)
        
    ## get svg path list for rendered out svg
    svg_paths = srh.generate_svg_path_list(os.path.join(sketch_dir,'svg'))    
    
    ## convert all svg to png
    srh.svg_to_png(svg_paths,base_dir=sketch_dir)    